In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/home/ldap/coccia@private.list.lu/oat_2024/MultimodalLLMs/MMLLMValidatorsTesting/data/batch_results/clean_evaluation_results.csv")

In [3]:
# Convert the "timestamp_token" column into datetime format
df["timestamp_token_dt"] = pd.to_datetime(df["timestamp_token"], unit="ms", errors="coerce")

# Show sample rows with the converted datetime
df[["timestamp_token", "timestamp_token_dt"]].head()


,timestamp_token,timestamp_token_dt
0,1.756713e+12,2025-09-01 07:54:53.832
1,1.756713e+12,2025-09-01 07:54:53.843
2,1.756713e+12,2025-09-01 07:54:53.845
3,1.756713e+12,2025-09-01 07:54:53.846
4,NaN,NaT


In [11]:
import json
import pandas as pd # Optional, but great for viewing data in a notebook

# --- Configuration ---
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.max_rows", 70)
file_path = '/home/ldap/coccia@private.list.lu/oat_2024/MultimodalLLMs/MMLLMValidatorsTesting/data/batch_request_file.results.jsonl' 

# --- Processing ---
false_validity_results = []

print(f"Reading file: {file_path}...")

# Open the JSON Lines file and process it line by line
with open(file_path, 'r') as f:
    for line in f:
        try:
            # 1. Load the main JSON object from the current line
            outer_data = json.loads(line)
            
            # 2. Navigate to the model's output text, which is a JSON string
            inner_json_string = outer_data['response']['body']['output'][0]['content'][0]['text']
            
            # 3. Load the inner JSON string to get the actual result
            inner_data = json.loads(inner_json_string)
            
            # 4. Check if the 'validity' key is False
            if not inner_data.get('validity', True): # Safely check for False
                result = {
                    'trait': inner_data.get('trait'),
                    'question_id': outer_data.get('custom_id').split("-")[-2],
                    'reasoning': inner_data.get('reasoning')
                }
                false_validity_results.append(result)

        except (json.JSONDecodeError, KeyError, IndexError) as e:
            # This will catch lines that are not valid JSON or have a different structure
            print(f"Skipping a malformed or unexpected line: {e}")

print(f"Done! Found {len(false_validity_results)} examples with false validity.")

# --- Display Results ---

# Option 1: Print the raw list of dictionaries
# print(false_validity_results)

# Option 2 (Recommended for Jupyter): Display as a clean table using pandas
if false_validity_results:
    df = pd.DataFrame(false_validity_results)
    display(df)
else:
    print("No examples with false validity were found.")

Reading file: /home/ldap/coccia@private.list.lu/oat_2024/MultimodalLLMs/MMLLMValidatorsTesting/data/batch_request_file.results.jsonl...
Done! Found 64 examples with false validity.


,trait,question_id,reasoning
0,Visual Clarity,42,"The Punnett square lacks clear labeling and context, making it difficult to understand the relationship between the genotypes and the expected phenotypic ratios."
1,Text-Image Coherence,42,"The text states that the expected ratio of offspring with a woolly fleece to those with a hairy fleece is 4:0, but the Punnett square shows only ff genotypes, which contradicts the expected outcome of having any hairy fleece offspring."
2,Visual Clarity,87,"The image contains dense foliage that may distract from the main focus of the Daintree rain forest ecosystem, making it harder to identify key features."
3,Visual Clarity,92,"The map lacks clear labeling of states and capitals, making it difficult to identify Delaware and its capital."
4,Text-Image Coherence,92,"The image is a map of the United States but does not highlight or indicate Delaware, making it difficult for students to connect the question about Delaware's capital with the visual information provided."
5,Visual Clarity,161,"The images of the robin and cane toad create confusion as they are presented alongside the ostrich, which is the correct answer, leading to potential distraction from the main focus."
6,Functional Relevance,234,The image does not provide essential information related to the question about Cooper's test; it appears to be decorative and does not aid in understanding the context.
7,Visual Clarity,234,The image contains unnecessary elements such as cluttered backgrounds and multiple objects that distract from the main focus of the design process.
8,Text-Image Coherence,234,"The image shows a person studying a design but does not provide any information related to the specific testing of the park designs mentioned in the question, leading to a lack of coherence."
9,Standard Presentation,237,"The magnets are shown in a non-standard orientation, which may confuse students about their poles and the resulting magnetic interaction."


In [12]:
df.to_csv("mistakes.csv")